# Simple WelDX File Processors

## Introduction
This tutorial is about applying functions from a dynamically loaded  loaded module on a weldx file:

* import module by name
* detect all functions
* let the user choose a function interactive for file manipulation
* print it as a json

1. First starting with the imports of weldx:

In [1]:
import asdf
import weldx
import weldx.asdf
from weldx import Q_ # pint quantity from the weldx package
from weldx.asdf.extension import WeldxAsdfExtension, WeldxExtension
from weldx.welding.groove.iso_9692_1 import get_groove, IsoBaseGroove, _create_test_grooves
from weldx.core import TimeSeries as TS
from weldx.welding.processes import GmawProcess

from inspect import isfunction, getmembers, getmodule
from typing import List

from ipywidgets import interact, interact_manual, fixed
from IPython.display import display, HTML, JSON, IFrame

In [2]:
# Get all functions for file manipulation of a specific module
weldx_processor_module = __import__("weldx_processors") # simulate dynamic module loading
weldx_processors = [m[1] for m in getmembers(weldx_processor_module) if isfunction(m[1]) and getmodule(m[1]) == weldx_processor_module]
# for a real use case, make some filtering for e.g. by analysing anotations
weldx_processor_options = { p.__name__:p for p in weldx_processors }
print(weldx_processor_options)

{'add_process_spray': <function add_process_spray at 0x000001E09F5A73A0>, 'add_pulse_ui': <function add_pulse_ui at 0x000001E0B2CDC670>, 'add_seam_length': <function add_seam_length at 0x000001E09F4C4820>}


In [3]:
# Create an initial file
example_groove = list(_create_test_grooves().values())[0][0]
tree = {"workpiece": {"groove": example_groove}}
file = asdf.AsdfFile(tree)
file_buffer = weldx.asdf.util.write_buffer(file.tree)
weldx.asdf.util.asdf_json_repr(file_buffer)

<IPython.core.display.JSON object>

In [4]:
# Let choose a user, that function he wants to use for file manipulation
def process_file(options, file):
    process_file = weldx_processor_options[options]
    new_file = process_file(file)
    new_file_buffer = weldx.asdf.util.write_buffer(new_file.tree)
    display(weldx.asdf.util.asdf_json_repr(new_file_buffer))
    
interact_manual(process_file, options=weldx_processor_options.keys(), file=fixed(file))

interactive(children=(Dropdown(description='options', options=('add_process_spray', 'add_pulse_ui', 'add_seam_…

<function __main__.process_file(options, file)>